In [ ]:
from transformers import pipeline

# Загружаем предобученный пайплайн Whisper
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3-turbo")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
model_id = "openai/whisper-large-v3-turbo"
chunk_duration = 30  # Длительность одной части в секундах
sampling_rate = 16000
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id)

In [ ]:
import librosa
import numpy as np
import torch

In [ ]:
def pipeline_transcribation(audio_path):
  y, sr = librosa.load(audio_path, sr=16000)

  # Длительность каждой части в секундах
  chunk_duration = 30
  chunk_size = sr * chunk_duration  # Размер части в сэмплах

  # Нарезаем аудио на части
  chunks = [y[i:i + chunk_size] for i in range(0, len(y), chunk_size)]

  transcriptions = []
  for i, chunk in enumerate(chunks):
    print(f"Обработка части {i + 1}/{len(chunks)}...")

    # Преобразуем аудио в мел-спектрограмму
    inputs = processor(chunk, sampling_rate=sampling_rate, return_tensors="pt")

    with torch.no_grad():
        generated_ids = model.generate(inputs.input_features)
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    transcriptions.append(transcription)

  full_transcription = " ".join(transcriptions)

  return full_transcription

In [ ]:
audio_paths = ["gorky-park-moscow-calling-2.mp3", "kino-peremen-3.mp3", "anime/Fire Force.mp3", "anime/Noragami.mp3"]

Пример транскрибации

In [ ]:
trs = []
for audio_path in audio_paths:
  print(audio_path)
  tr = pipeline_transcribation(audio_path)
  trs.append(tr)
  print(tr)

gorky-park-moscow-calling-2.mp3
Обработка части 1/2...
Обработка части 2/2...
 Yeah! Yeah! Yeah! Pascal Collin!  Oh, this is too busy now.
kino-peremen-3.mp3
Обработка части 1/2...
Обработка части 2/2...
 Перемь  И в наших слезах импульсации лет, Горякет.
anime/Fire Force.mp3
Обработка части 1/8...
Обработка части 2/8...
Обработка части 3/8...
Обработка части 4/8...
Обработка части 5/8...
Обработка части 6/8...
Обработка части 7/8...
Обработка части 8/8...
The Love is my enemy 僕らは歩き慣れてきた日々も問うた夢は安泰な暮らしだが 刺激ブース キュエにタラタラ照らす 歩み方の歩き慣れていた道はどこだ?時はたまにじゃったが温もりに包まれたがほむらがあった シルベの方へ思い出すは 優しいメロディ永遠繋いだと 繋いだとよ 誰も満タイなと笑わせみる輝けばいつかは光もさえ僕らは命の火が消えるその日まで歩いてゆくところでなぜ僕らは思考を急にやめているんだ夢は安泰な暮らしだが知識不足 食性分やみを薄金 下がりついたまま朽ちていくんだここはこう歌う中だが 傷跡がひりつきたな水面があったつ光の方へ 手を取るは新しいメモリー夜空が分かつ シルベの方へ 遮るは堅苦しい背景 永遠はないんだとないんだと言うやっぱ苦しいなと泣いてみる風船のいつかは死ぼむが割れる僕らは命の泉を守り続けて繋いでいけ 伝えきれずに視線遊びきれずに決めつめないのくさい映画が地獄じゃある前して音が出る玩具も痛めを飛ばす魔法も全部僕にとっての宝物永遠は何だと泣いたそういう 命の火が消えるその日まで歩いてゆく  you
anime/Noragami.mp3
Обработка части 1/9...
Обработка части 2/9...
Об